In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install netCDF4
!pip install xarray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Imports

In [3]:
# IMPORTS

import numpy as np
import netCDF4 as nc
import scipy.io as sio
import xarray as xr
import pickle
from scipy.stats import pearsonr
import plotly
import tensorflow
import keras.backend as K
#from data_manager import ClutteredMNIST
#from visualizer import plot_mnist_sample
#from visualizer import print_evaluation
#from visualizer import plot_mnist_grid
import netCDF4
import numpy as np
from keras.layers import Input, Convolution2D, Convolution1D, MaxPooling2D, Dense, Dropout, \
                          Flatten, concatenate, Activation, Reshape, \
                          UpSampling2D,ZeroPadding2D
import keras
from keras.callbacks import History
history = History()

import keras
from keras.layers import Conv2D, Conv2DTranspose, Cropping2D, Concatenate, ZeroPadding2D
from keras.models import load_model
from keras.layers import Input
from keras.models import Model
from keras.layers import Activation
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

# from utils import get_initial_weights
# from layers import BilinearInterpolation

__version__ = 0.1 

In [4]:
def get_initial_weights(output_size):
    b = np.zeros((2, 3), dtype='float32')
    b[0, 0] = 1
    b[1, 1] = 1
    W = np.zeros((output_size, 6), dtype='float32')
    weights = [W, b.flatten()]
    return weights

In [5]:
import keras.backend as K
#from keras.engine.topology import Layer
from tensorflow.python.keras.layers import Layer

if K.backend() == 'tensorflow':
    import tensorflow as tf

    def K_meshgrid(x, y):
        return tf.meshgrid(x, y)

    def K_linspace(start, stop, num):
        return tf.linspace(start, stop, num)

else:
    raise Exception("Only 'tensorflow' is supported as backend")


class BilinearInterpolation(Layer):
    """Performs bilinear interpolation as a keras layer
    References
    ----------
    [1]  Spatial Transformer Networks, Max Jaderberg, et al.
    [2]  https://github.com/skaae/transformer_network
    [3]  https://github.com/EderSantana/seya
    """

    def __init__(self, output_size, **kwargs):
        self.output_size = output_size
        super(BilinearInterpolation, self).__init__(**kwargs)

    def get_config(self):
        return {
            'output_size': self.output_size,
        }

    def compute_output_shape(self, input_shapes):
        height, width = self.output_size
        num_channels = input_shapes[0][-1]
        return (None, height, width, num_channels)

    def call(self, tensors, mask=None):
        X, transformation = tensors
        output = self._transform(X, transformation, self.output_size)
        return output

    def _interpolate(self, image, sampled_grids, output_size):

        batch_size = K.shape(image)[0]
        height = K.shape(image)[1]
        width = K.shape(image)[2]
        num_channels = K.shape(image)[3]

        x = K.cast(K.flatten(sampled_grids[:, 0:1, :]), dtype='float32')
        y = K.cast(K.flatten(sampled_grids[:, 1:2, :]), dtype='float32')

        x = .5 * (x + 1.0) * K.cast(width, dtype='float32')
        y = .5 * (y + 1.0) * K.cast(height, dtype='float32')

        x0 = K.cast(x, 'int32')
        x1 = x0 + 1
        y0 = K.cast(y, 'int32')
        y1 = y0 + 1

        max_x = int(K.int_shape(image)[2] - 1)
        max_y = int(K.int_shape(image)[1] - 1)

        x0 = K.clip(x0, 0, max_x)
        x1 = K.clip(x1, 0, max_x)
        y0 = K.clip(y0, 0, max_y)
        y1 = K.clip(y1, 0, max_y)

        pixels_batch = K.arange(0, batch_size) * (height * width)
        pixels_batch = K.expand_dims(pixels_batch, axis=-1)
        flat_output_size = output_size[0] * output_size[1]
        base = K.repeat_elements(pixels_batch, flat_output_size, axis=1)
        base = K.flatten(base)

        # base_y0 = base + (y0 * width)
        base_y0 = y0 * width
        base_y0 = base + base_y0
        # base_y1 = base + (y1 * width)
        base_y1 = y1 * width
        base_y1 = base_y1 + base

        indices_a = base_y0 + x0
        indices_b = base_y1 + x0
        indices_c = base_y0 + x1
        indices_d = base_y1 + x1

        flat_image = K.reshape(image, shape=(-1, num_channels))
        flat_image = K.cast(flat_image, dtype='float32')
        pixel_values_a = K.gather(flat_image, indices_a)
        pixel_values_b = K.gather(flat_image, indices_b)
        pixel_values_c = K.gather(flat_image, indices_c)
        pixel_values_d = K.gather(flat_image, indices_d)

        x0 = K.cast(x0, 'float32')
        x1 = K.cast(x1, 'float32')
        y0 = K.cast(y0, 'float32')
        y1 = K.cast(y1, 'float32')

        area_a = K.expand_dims(((x1 - x) * (y1 - y)), 1)
        area_b = K.expand_dims(((x1 - x) * (y - y0)), 1)
        area_c = K.expand_dims(((x - x0) * (y1 - y)), 1)
        area_d = K.expand_dims(((x - x0) * (y - y0)), 1)

        values_a = area_a * pixel_values_a
        values_b = area_b * pixel_values_b
        values_c = area_c * pixel_values_c
        values_d = area_d * pixel_values_d
        return values_a + values_b + values_c + values_d

    def _make_regular_grids(self, batch_size, height, width):
        # making a single regular grid
        x_linspace = K_linspace(-1., 1., width)
        y_linspace = K_linspace(-1., 1., height)
        x_coordinates, y_coordinates = K_meshgrid(x_linspace, y_linspace)
        x_coordinates = K.flatten(x_coordinates)
        y_coordinates = K.flatten(y_coordinates)
        ones = K.ones_like(x_coordinates)
        grid = K.concatenate([x_coordinates, y_coordinates, ones], 0)

        # repeating grids for each batch
        grid = K.flatten(grid)
        grids = K.tile(grid, K.stack([batch_size]))
        return K.reshape(grids, (batch_size, 3, height * width))

    def _transform(self, X, affine_transformation, output_size):
        batch_size, num_channels = K.shape(X)[0], K.shape(X)[3]
        transformations = K.reshape(affine_transformation,
                                    shape=(batch_size, 2, 3))
        # transformations = K.cast(affine_transformation[:, 0:2, :], 'float32')
        regular_grids = self._make_regular_grids(batch_size, *output_size)
        sampled_grids = K.batch_dot(transformations, regular_grids)
        interpolated_image = self._interpolate(X, sampled_grids, output_size)
        new_shape = (batch_size, output_size[0], output_size[1], num_channels)
        interpolated_image = K.reshape(interpolated_image, new_shape)
        return interpolated_image


### Model

In [6]:
## border_mode --> padding

def stn(input_shape=(32, 64, 1), sampling_size=(8, 16), num_classes=10):
    inputs = Input(shape=input_shape)
    conv1 = Conv2D(32, (5, 5), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2,2), padding='same')(conv1)

    conv2 = Conv2D(32, (5, 5), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2,2), padding='same')(conv2)

    conv3 = Conv2D(32, (5, 5), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv3)


    conv5 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv3)
    conv5 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv5)
    
    locnet = Flatten()(conv5)
    locnet = Dense(500)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(200)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(100)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(50)(locnet)
    locnet = Activation('relu')(locnet)
    weights = get_initial_weights(50)
    locnet = Dense(6, weights=weights)(locnet)
    x = BilinearInterpolation(sampling_size)([inputs, locnet])


    up6 = keras.layers.Concatenate(axis=-1)([Conv2D(32, (2, 2), activation='relu', padding='same')(UpSampling2D(size=(2, 2))(x)), conv2])
    conv6 = Conv2D(32, (5, 5), activation='relu', padding='same')(up6)
    conv6 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv6)

    up7 = keras.layers.Concatenate(axis=-1)([Conv2D(32, (2, 2), activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6)), conv1])
    conv7 = Conv2D(32, (5, 5), activation='relu', padding='same')(up7)
    conv7 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv7)

    conv10 = Conv2D(1, (5, 5), activation='linear',padding='same')(conv7)
    model = Model(inputs=inputs, outputs=conv10)
    return model

In [ ]:
def unet_baseline(input_shape=(32, 64, 1), sampling_size=(8, 16), num_classes=10):
    inputs = Input(shape=input_shape)
    conv1 = Conv2D(32, (5, 5), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(32, (5, 5), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(32, (5, 5), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv3)

    conv5 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv3)
    x = Conv2D(32, (5, 5), activation='relu', padding='same')(conv5)

    up6 = keras.layers.Concatenate(axis=-1)([Conv2D(32, (2, 2), activation='relu', padding='same')(UpSampling2D(size=(2, 2))(x)), conv2])
    conv6 = Conv2D(32, (5, 5), activation='relu', padding='same')(up6)
    conv6 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv6)

    up7 = keras.layers.Concatenate(axis=-1)([Conv2D(32, (2, 2),activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6)), conv1])
    conv7 = Conv2D(32, (5, 5), activation='relu', padding='same')(up7)
    conv7 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv7)

    conv10 = Conv2D(1, (5, 5), activation='linear', padding='same')(conv7)

    model = Model(inputs=inputs, outputs=conv10)

    return model

In [ ]:
def unet_old(input_shape=(32, 64, 1), sampling_size=(8, 16), num_classes=10):
  
    inputs = Input(shape=(32, 64, 1))

    conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(32, 3, activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(32, 3, activation='relu', padding='same')(pool2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    locnet = Flatten()(pool3)
    locnet = Dense(500)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(250)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(50)(locnet)
    locnet = Activation('relu')(locnet)
    weights = get_initial_weights(50)
    locnet = Dense(6, weights=weights)(locnet)
    x = BilinearInterpolation((8, 16))([inputs, locnet])

    conv4 = Conv2D(32, (5, 5), activation='relu', padding='same')(x)
    up4 = UpSampling2D(size=(2, 2))(conv4)

    conv5 = Conv2D(32, (5, 5), activation='relu', padding='same')(up4)
    up5 = UpSampling2D(size=(2, 2))(conv5)

    conv6 = Conv2D(1, (5, 5), activation='relu', padding='same')(up5)

    model = Model(inputs=inputs, outputs=conv6)

    return model

In [7]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [ ]:
# N = 10
# gamma = 0.1
# def L1_Loss(y_true, y_pred):
#   loss = (1-gamma)*(tf.math.reduce_sum((y_pred - y_true)**2)) + (gamma*tf.math.reduce_sum(y_pred))
#   return loss/N

In [ ]:
gamma = 0.01
def L1_Loss(y_true, y_pred):
  term1 = (1 - gamma) * tf.reduce_mean(tf.square(y_pred - y_true))
  term2 = gamma * tf.reduce_mean(y_pred)
  L1 = term1 + term2

  return L1

In [ ]:
N = 10
gamma = 0.25
def L2_Loss(y_true,y_pred):
    diff_pred = y_pred[1:] - y_pred[:-1]
    diff_true = y_true[1:] - y_true[:-1]

    # Define Ψ(pred) and Ψ(true)
    psi_pred = tf.math.sigmoid(y_pred)
    psi_true = tf.math.sigmoid(y_true)

    # Calculate first-order difference of Ψ(pred) and Ψ(true)
    psi_diff_pred = psi_pred[1:] - psi_pred[:-1]
    psi_diff_true = psi_true[1:] - psi_true[:-1]

    # Calculate L2
    term1 = (1 - gamma) * tf.reduce_mean(tf.square(psi_pred - psi_true))
    term2 = gamma * tf.reduce_mean(tf.square(psi_diff_pred - psi_diff_true))
    L2 = term1 + term2

    return L2/N

In [ ]:
model = unet_old()
model.compile(loss=root_mean_squared_error, optimizer='adam')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 64, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 64, 32)   320         ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 16, 32, 32)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 16, 32, 32)   9248        ['max_pooling2d[0][0]']          
                                                                                              

In [ ]:
model = unet_baseline()
model.compile(loss=root_mean_squared_error, optimizer='adam')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 64, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 64, 32)   832         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 32, 64, 32)   25632       ['conv2d[0][0]']                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 16, 32, 32)   0           ['conv2d_1[0][0]']               
                                                                                              

In [8]:
model = stn()
model.compile(loss=root_mean_squared_error, optimizer='adam')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 64, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 64, 32)   832         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 32, 64, 32)   25632       ['conv2d[0][0]']                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 16, 32, 32)   0           ['conv2d_1[0][0]']               
                                                                                              

In [9]:
# Getting back the objects:
with open('/content/drive/MyDrive/DDWP/z500_mean_sdev.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    M, sdev = pickle.load(f)

M, sdev

(54110.977, 3355.03)

### Prediction Loop

In [10]:
model.load_weights('/content/drive/MyDrive/DDWP/stn_rmse.h5')

In [11]:
### This is the autoregressive prediction with baseline U-NET without DA ####
# /content/drive/MyDrive/DDWP/geopotential_z500/geopotential_500hPa_2018_5.625deg.nc
lead = 12
path = "/content/drive/MyDrive/DDWP/geopotential_z500/geopotential_500hPa_2018_5.625deg.nc"
F=nc.Dataset(path)
Z=F['z']
Z=(Z-M)/sdev

testN=1000
x_test=Z[0:testN,:,:]
x_test=x_test.reshape([np.size(x_test,0),32,64,1])
y_test=Z[lead:testN+lead,:,:]
y_test=y_test.reshape([np.size(y_test,0),32,64,1])

pred=np.zeros([testN,32,64,1])
for k in range (0, testN):
    if(k==0):
      pred[k,:,:,0]=model.predict(x_test[k,:,:,0].reshape([1,32,64,1])).reshape([32,64])
    else:
      pred[k,:,:,0]=model.predict(pred[k-1,:,:,0].reshape([1,32,64,1])).reshape([32,64])

1/1 [==============================] - 0s 45ms/step


In [12]:
print("Prediction done, saving to file")
#save preds
#with open(var+'_noSTN_lead'+str(lead)+'_preds.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
with open('/content/drive/MyDrive/DDWP/stn_rmse_2018_preds.pkl', 'wb') as f:  # Python 3: open(..., 'wb') 
    pickle.dump([pred, y_test], f)
print('Finished writing File')

Prediction done, saving to file
Finished writing File


### Generate Plots

In [14]:
print("Prediction done, saving to file")
#save preds
#with open(var+'_noSTN_lead'+str(lead)+'_preds.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
with open('/content/drive/MyDrive/DDWP/stn_preds.pkl', 'rb') as f:  # Python 3: open(..., 'wb') 
    pred, y_test = pickle.load(f)
print('loaded preds')

Prediction done, saving to file
loaded preds


In [ ]:
testN=1000

In [ ]:
print(pred.shape, y_test.shape)
x_pred = pred.reshape(testN, -1)
x_true = y_test.reshape(testN, -1)
print(x_pred.shape, x_true.shape)

(1000, 32, 64, 1) (1000, 32, 64, 1)
(1000, 2048) (1000, 2048)


In [ ]:
pred = pred.reshape(1000, 32, 64)
print(pred.shape)
print(y_test.squeeze().shape)

(1000, 32, 64)
(1000, 32, 64)


In [ ]:
import plotly.graph_objects as go
# define the colorscale with discrete colors

colorscale = [[0.0, 'rgb(41, 120, 255)'],
              [0.25, 'rgb(153, 204, 255)'],
              [0.5, 'rgb(255, 255, 255)'],
              [0.75, 'rgb(255, 204, 204)'],
              [1.0, 'rgb(255, 80, 80)']]
# create the heatmap trace
heatmap = go.Heatmap(z=y_test.squeeze()[5], colorscale=colorscale)

# create the layout and figure
layout = go.Layout(template="plotly_white", width = 400, height = 400,
                  title = 'Real z500 values - Day 5',
                  xaxis_title = "Longitude",
                  yaxis_title = "Latitude")
fig = go.Figure(data=[heatmap], layout=layout)

# display the figure
fig.show()

In [ ]:
import plotly.graph_objects as go
# Create figure
fig = go.Figure()
fig = go.Figure(data=go.Heatmap(z=pred[10]))

fig.update_layout(template="plotly_white", width = 400, height = 400,
                  title = 'z500 from UNET2 with L1 loss 4 after 10 days',
                  xaxis_title = "Longitude",
                  yaxis_title = "Latitude"
                  )

fig.show()

In [ ]:
print(np.isnan(x_pred).any())
print(np.isnan(x_true).any())
print(np.count_nonzero(np.isnan(x_pred)))
print(np.isinf(x_pred).any())
print(np.isinf(x_true).any())

False
False
0
False
False


In [ ]:
temp = x_pred
x_pred[~np.isfinite(temp)] = 0
x_true[~np.isfinite(temp)] = 0

In [ ]:
corrs = []
for x_predi, x_truei in zip(x_pred, x_true):
    corr = np.corrcoef(x_predi, x_truei)
    corrs.append(corr)
corrs_bucket = [np.mean(corrs[12*i:12*i+12]) for i in range(0, 83)]

In [ ]:
import plotly
plotly.__version__

'5.13.1'

In [ ]:
pip install plotly==5.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 13.8 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


In [ ]:
!pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.9 MB/s eta 0:00:00


In [ ]:
print(corrs_bucket[:20])

[0.9830736181790612, 0.9620870769814527, 0.9577788645289996, 0.9483448034134722, 0.9564112113296281, 0.9495522877175259, 0.945098592607832, 0.9410000104427064, 0.9356641007294852, 0.9302068475461339, 0.9218969671067746, 0.9129586700600809, 0.891351250632001, 0.8612294235920442, 0.8563205171250908, 0.8480221049587383, 0.8347796021253058, 0.8463495995830956, 0.8625565847569706, 0.8565353549693819]


In [ ]:
df = pd.DataFrame(corrs_bucket[:20])
df

In [ ]:
import plotly.express as px 
  
# Creating the Figure instance
fig = px.line(x=np.arange(0, 10, 0.5), y=corrs_bucket[:20], width=600, height=600)
fig.update_xaxes(range=[0,10])
fig.update_yaxes(range=[0.5,1])
fig.update_layout(
    title='baseline model',
    xaxis_title="Days",
    yaxis_title="R (correlation between actual and predicted)",
)
# showing the plot
fig.show()

#fig.write_image("/content/drive/MyDrive/DDWP/new_L2.png")